In [105]:
import pandas as pd   
import time
from sklearn.model_selection import train_test_split #, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [106]:
import os
print(os.getcwd())
#change working directory to the location of the data file
os.chdir('/mnt/d/Sajjad/08-2023/Python Code/Introduction to Machine Learning/')
print(os.getcwd())

/mnt/d/Sajjad/08-2023/Python Code/Introduction to Machine Learning
/mnt/d/Sajjad/08-2023/Python Code/Introduction to Machine Learning


Extracting data from UCI Machine Learning Repository

In [107]:
from ucimlrepo import fetch_ucirepo 

In [108]:
# fetch dataset 
cdc_diabetes_health_indicators = fetch_ucirepo(id=891) 
  
# data (as pandas dataframes) 
X = cdc_diabetes_health_indicators.data.features 
y = cdc_diabetes_health_indicators.data.targets 
  
# metadata 
print(cdc_diabetes_health_indicators.metadata) 
  
# variable information 
print(cdc_diabetes_health_indicators.variables) 

{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

Since Education and Income are in ordinal scale, hence, no need to do one-hot encoding. 

In [ ]:
#set Education Variable as Categorical
#X['Education'] = X['Education'].astype('category')
#X['Education'].value_counts()

In [109]:
X.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1,1,1,40,1,0,0,0,0,1,...,1,0,5,18,15,1,0,9,4,3
1,0,0,0,25,1,0,0,1,0,0,...,0,1,3,0,0,0,0,7,6,1
2,1,1,1,28,0,0,0,0,1,0,...,1,1,5,30,30,1,0,9,4,8
3,1,0,1,27,0,0,0,1,1,1,...,1,0,2,0,0,0,0,11,3,6
4,1,1,1,24,0,0,0,1,1,1,...,1,0,2,3,0,0,0,11,5,4


In [ ]:
X.dtypes

Making two train/test dataset. One without scaling and the other with scaling. 

In [ ]:
#without scaling
trainX2, testX2, trainy2, testy2 = train_test_split(X, y, test_size=0.3, random_state=2)

In [ ]:
from sklearn.preprocessing import StandardScaler

with scaling dataset

In [110]:
#scaled X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
trainX, testX, trainy, testy = train_test_split(X_scaled, y, test_size=0.3, random_state=2)

In [111]:
#count the distribution of y in train and test
print(trainy.value_counts())
testy.value_counts()

Diabetes_binary
0                  152917
1                   24659
Name: count, dtype: int64


Diabetes_binary
0                  65417
1                  10687
Name: count, dtype: int64

In [112]:
trainX.shape

(177576, 21)

In [113]:
testX.shape

(76104, 21)

this type casting is not always required but at times torch generates an error so just as a matter of caution converting all types to float32

In [114]:
#convert all int64 to float32
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainy = trainy.astype('float32')
testy = testy.astype('float32')

using "torch" as keras_backend. Could have used tensorflow or jax as well.

In [115]:
import torch
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense, Dropout

explore batch size, iteration size

In [120]:
# get the number of input features in X and assign to n_features
n_features = trainX.shape[1]
# Create a Sequential model
model = Sequential()

# Add the first hidden layer with 10 neurons and specify the input shape
model.add(Dense(10, input_dim=n_features, activation='relu'))

# Add the second hidden layer with 5 neurons
model.add(Dense(5, activation='relu'))

# Add the output layer with 1 neuron (for binary classification) and 'sigmoid' activation
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC']) #change optimizer

# Print the model summary
print(model.summary())


Model: "sequential_28"


_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_82 (Dense)            (None, 10)                220       
                                                                 
 dense_83 (Dense)            (None, 5)                 55        
                                                                 
 dense_84 (Dense)            (None, 1)                 6         
                                                                 
Total params: 281 (1.10 KB)
Trainable params: 281 (1.10 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [121]:
model.fit(trainX, trainy, epochs=20, batch_size=512)

Epoch 1/20


347/347 [==============================] - 4s 8ms/step - loss: 0.4434 - auc: 0.6517
Epoch 2/20
347/347 [==============================] - 3s 8ms/step - loss: 0.3227 - auc: 0.8133
Epoch 3/20
347/347 [==============================] - 3s 8ms/step - loss: 0.3185 - auc: 0.8199
Epoch 4/20
347/347 [==============================] - 3s 8ms/step - loss: 0.3164 - auc: 0.8229
Epoch 5/20
347/347 [==============================] - 3s 8ms/step - loss: 0.3151 - auc: 0.8248
Epoch 6/20
347/347 [==============================] - 3s 8ms/step - loss: 0.3143 - auc: 0.8261
Epoch 7/20
347/347 [==============================] - 3s 8ms/step - loss: 0.3137 - auc: 0.8270
Epoch 8/20
347/347 [==============================] - 3s 8ms/step - loss: 0.3133 - auc: 0.8276
Epoch 9/20
347/347 [==============================] - 3s 8ms/step - loss: 0.3130 - auc: 0.8280
Epoch 10/20
347/347 [==============================] - 3s 8ms/step - loss: 0.3127 - auc: 0.8284
Epoch 11/20
347/347 [==============================] - 3s 8m

In [122]:
model.evaluate(testX, testy, batch_size=4096)

19/19 [==============================] - 0s 10ms/step - loss: 0.3147 - auc: 0.8287


[0.3146805167198181, 0.8286526799201965]

comparing the performance with catboost, xgboost, and lgbm

In [123]:
from catboost import CatBoostClassifier
import xgboost as xgb
import lightgbm as lgb

In [124]:
num_of_models = 100
depth_level = 3

In [125]:
#use lgboost
lgb_model = lgb.LGBMClassifier(max_depth=depth_level, n_estimators=num_of_models, learning_rate=0.1)
start_time = time.time()
#fit xgb_model
lgb_model.fit(trainX2,trainy2)
md_probs = lgb_model.predict_proba(testX2)
md_probs = md_probs[:,1]
md_auc = roc_auc_score(testy2, md_probs)
print("LG Boost", " : ", md_auc)
#record the end time
end_time = time.time()
#calculate the total time
total_time = end_time - start_time
print("Total time LGB: ", total_time)

[LightGBM] [Info] Number of positive: 24659, number of negative: 152917
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007048 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 202
[LightGBM] [Info] Number of data points in the train set: 177576, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.138864 -> initscore=-1.824753
[LightGBM] [Info] Start training from score -1.824753
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
LG Boost  :  0.8296622740763209
Total time LGB:  0.46257662773132324


In [126]:
cb = CatBoostClassifier(iterations=num_of_models, depth=depth_level, learning_rate=0.1, loss_function='Logloss', verbose=False)
#record the start time
start_time = time.time()
cb.fit(trainX2,trainy2)
md_probs = cb.predict_proba(testX2)
md_probs = md_probs[:,1]
md_auc = roc_auc_score(testy2, md_probs)
print("Cat Boost", " : ", md_auc)
#record the end time
end_time = time.time()
#calculate the total time
total_time = end_time - start_time
print("Total time CB: ", total_time)

Cat Boost  :  0.8289070111234721
Total time CB:  1.9635086059570312


In [127]:
#use xgboost
xgb_model = xgb.XGBClassifier(max_depth=depth_level, n_estimators=num_of_models, learning_rate=0.1)
start_time = time.time()
#fit xgb_model
xgb_model.fit(trainX2,trainy2)
md_probs = xgb_model.predict_proba(testX2)
md_probs = md_probs[:,1]
md_auc = roc_auc_score(testy2, md_probs)
print("XG Boost", " : ", md_auc)
#record the end time
end_time = time.time()
#calculate the total time
total_time = end_time - start_time
print("Total time XGB: ", total_time)

XG Boost  :  0.8295503663157561
Total time XGB:  1.6627285480499268


using sklearn version of feedforward neural network (also called Multi-layer Perceptron)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [128]:
#using multilayer perceptron of sklearn
mlp = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=10, solver='adam', verbose=10, random_state=1,
                    learning_rate_init=.1, batch_size=2048)
#mlp.fit(trainX, trainy)
for epoch in range(10):  # Set the desired number of epochs
    mlp.partial_fit(trainX, trainy, classes=[0, 1])

    # Evaluate on the validation set and print AUC ROC
    y_prob = mlp.predict_proba(testX)[:, 1]
    auc_roc = roc_auc_score(testy, y_prob)
    print(f"Epoch {epoch + 1}, AUC ROC: {auc_roc:.4f}") 

Iteration 1, loss = 0.35431792
Epoch 1, AUC ROC: 0.8236
Iteration 1, loss = 0.31578031
Epoch 2, AUC ROC: 0.8266
Iteration 1, loss = 0.31479602
Epoch 3, AUC ROC: 0.8269
Iteration 1, loss = 0.31466764
Epoch 4, AUC ROC: 0.8270
Iteration 1, loss = 0.31458664
Epoch 5, AUC ROC: 0.8268
Iteration 1, loss = 0.31459864
Epoch 6, AUC ROC: 0.8272
Iteration 1, loss = 0.31455789
Epoch 7, AUC ROC: 0.8274
Iteration 1, loss = 0.31444263
Epoch 8, AUC ROC: 0.8276
Iteration 1, loss = 0.31438872
Epoch 9, AUC ROC: 0.8272
Iteration 1, loss = 0.31453122
Epoch 10, AUC ROC: 0.8279


In [129]:
from keras.optimizers import SGD

experimenting with different optimizers

In [130]:
# get the number of input features in X and assign to n_features
n_features = trainX.shape[1]
# Create a Sequential model
model = Sequential()

# Add the first hidden layer with 10 neurons and specify the input shape
model.add(Dense(20, input_dim=n_features, activation='relu'))

# Add the second hidden layer with 5 neurons
model.add(Dense(10, activation='relu'))

# Add the output layer with 1 neuron (for binary classification) and 'sigmoid' activation
model.add(Dense(1, activation='sigmoid'))

# Create an SGD optimizer with momentum
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9)
# Compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC']) #change optimizer
model.compile(optimizer=sgd_optimizer, loss='binary_crossentropy', metrics=['AUC'])

model.fit(trainX, trainy, epochs=20, batch_size=2048)


Epoch 1/20
87/87 [==============================] - 1s 9ms/step - loss: 0.3838 - auc: 0.6940
Epoch 2/20
87/87 [==============================] - 1s 8ms/step - loss: 0.3300 - auc: 0.7994
Epoch 3/20
87/87 [==============================] - 1s 7ms/step - loss: 0.3253 - auc: 0.8078
Epoch 4/20
87/87 [==============================] - 1s 6ms/step - loss: 0.3230 - auc: 0.8115
Epoch 5/20
87/87 [==============================] - 1s 6ms/step - loss: 0.3215 - auc: 0.8140
Epoch 6/20
87/87 [==============================] - 1s 6ms/step - loss: 0.3204 - auc: 0.8158
Epoch 7/20
87/87 [==============================] - 1s 9ms/step - loss: 0.3196 - auc: 0.8173
Epoch 8/20
87/87 [==============================] - 1s 8ms/step - loss: 0.3189 - auc: 0.8185
Epoch 9/20
87/87 [==============================] - 1s 9ms/step - loss: 0.3182 - auc: 0.8194
Epoch 10/20
87/87 [==============================] - 1s 9ms/step - loss: 0.3177 - auc: 0.8203
Epoch 11/20
87/87 [==============================] - 1s 9ms/step - lo

In [131]:
model.evaluate(testX, testy, verbose=0, batch_size=2048)

[0.3177793323993683, 0.8239396810531616]

In [132]:
from keras.callbacks import EarlyStopping

experimenting with early stopping without validation set

In [133]:
# Define EarlyStopping callback to monitor training loss
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

# Train the model with dropout and early stopping
model.fit(trainX, trainy, epochs=10, batch_size=2048,callbacks=[early_stopping])

Epoch 1/10
87/87 [==============================] - 1s 10ms/step - loss: 0.3145 - auc: 0.8253
Epoch 2/10
87/87 [==============================] - 1s 8ms/step - loss: 0.3144 - auc: 0.8255
Epoch 3/10
87/87 [==============================] - 1s 9ms/step - loss: 0.3142 - auc: 0.8257
Epoch 4/10
87/87 [==============================] - 1s 8ms/step - loss: 0.3141 - auc: 0.8260
Epoch 5/10
87/87 [==============================] - 1s 7ms/step - loss: 0.3140 - auc: 0.8262
Epoch 6/10
87/87 [==============================] - 1s 6ms/step - loss: 0.3138 - auc: 0.8263
Epoch 7/10
87/87 [==============================] - 1s 7ms/step - loss: 0.3137 - auc: 0.8265
Epoch 8/10
87/87 [==============================] - 1s 9ms/step - loss: 0.3136 - auc: 0.8267
Epoch 9/10
87/87 [==============================] - 1s 9ms/step - loss: 0.3135 - auc: 0.8268
Epoch 10/10
87/87 [==============================] - 1s 9ms/step - loss: 0.3134 - auc: 0.8270


In [ ]:
model.evaluate(testX, testy, verbose=0, batch_size=2048)

experimenting with Dropout

In [ ]:
from keras.layers import Dropout

In [134]:
# Create a Sequential model
model = Sequential()

# Add layers to the model (example architecture with dropout)
model.add(Dense(20, input_dim=21, activation='relu'))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.5
model.add(Dense(15, activation='relu'))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.3
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.3
model.add(Dense(1, activation='sigmoid'))
# Create an SGD optimizer with momentum
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC']) #change optimizer
model.fit(trainX, trainy, epochs=30, batch_size=2048, callbacks=[early_stopping])

Epoch 1/30
87/87 [==============================] - 2s 11ms/step - loss: 0.5168 - auc: 0.5813
Epoch 2/30
87/87 [==============================] - 1s 7ms/step - loss: 0.3839 - auc: 0.7148
Epoch 3/30
87/87 [==============================] - 1s 7ms/step - loss: 0.3639 - auc: 0.7530
Epoch 4/30
87/87 [==============================] - 1s 7ms/step - loss: 0.3544 - auc: 0.7693
Epoch 5/30
87/87 [==============================] - 1s 8ms/step - loss: 0.3493 - auc: 0.7764
Epoch 6/30
87/87 [==============================] - 1s 8ms/step - loss: 0.3465 - auc: 0.7806
Epoch 7/30
87/87 [==============================] - 1s 7ms/step - loss: 0.3432 - auc: 0.7862
Epoch 8/30
87/87 [==============================] - 1s 8ms/step - loss: 0.3412 - auc: 0.7887
Epoch 9/30
87/87 [==============================] - 1s 9ms/step - loss: 0.3390 - auc: 0.7932
Epoch 10/30
87/87 [==============================] - 1s 8ms/step - loss: 0.3384 - auc: 0.7936
Epoch 11/30
87/87 [==============================] - 1s 7ms/step - l

In [135]:
# Create a Sequential model
model = Sequential()

# Add layers to the model (example architecture with dropout)
model.add(Dense(20, input_dim=21, activation='relu'))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.5
model.add(Dense(1, activation='sigmoid'))
# Create an SGD optimizer with momentum
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC']) #change optimizer
model.fit(trainX, trainy, epochs=30, batch_size=2048, callbacks=[early_stopping])

Epoch 1/150
87/87 [==============================] - 1s 9ms/step - loss: 0.6265 - auc: 0.5015
Epoch 2/150
87/87 [==============================] - 1s 8ms/step - loss: 0.4080 - auc: 0.6938
Epoch 3/150
87/87 [==============================] - 1s 9ms/step - loss: 0.3564 - auc: 0.7626
Epoch 4/150
87/87 [==============================] - 1s 8ms/step - loss: 0.3431 - auc: 0.7820
Epoch 5/150
87/87 [==============================] - 1s 8ms/step - loss: 0.3370 - auc: 0.7918
Epoch 6/150
87/87 [==============================] - 1s 7ms/step - loss: 0.3336 - auc: 0.7976
Epoch 7/150
87/87 [==============================] - 1s 7ms/step - loss: 0.3315 - auc: 0.8012
Epoch 8/150
87/87 [==============================] - 1s 6ms/step - loss: 0.3287 - auc: 0.8053
Epoch 9/150
87/87 [==============================] - 1s 6ms/step - loss: 0.3272 - auc: 0.8078
Epoch 10/150
87/87 [==============================] - 1s 8ms/step - loss: 0.3260 - auc: 0.8093
Epoch 11/150
87/87 [==============================] - 1s 9m

KeyboardInterrupt: 

experimenting with weights regularization

In [136]:
from keras.regularizers import l1, l2

In [ ]:
# Create a Sequential model
model = Sequential()

# Add layers to the model (example architecture with dropout)
model.add(Dense(20, input_dim=21, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.5
model.add(Dense(10, input_dim=21, activation='relu', kernel_regularizer=l1(0.01)))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.5
model.add(Dense(1, activation='sigmoid'))
# Create an SGD optimizer with momentum
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC']) #change optimizer
model.fit(trainX, trainy, epochs=150, batch_size=2048, callbacks=[early_stopping])

experimenting with early stopping with validation set

In [137]:
valX, trainX3, valy, trainy3 = train_test_split(trainX, trainy, test_size=0.2, random_state=42)

In [138]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# Create a Sequential model
model = Sequential()

# Add layers to the model (example architecture with dropout)
model.add(Dense(20, input_dim=21, activation='relu'))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.5
model.add(Dense(10, input_dim=21, activation='relu'))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.5
model.add(Dense(1, activation='sigmoid'))
# Create an SGD optimizer with momentum
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC']) #change optimizer
model.fit(trainX3, trainy3, validation_data=(valX, valy), epochs=50, batch_size=2048, callbacks=[early_stopping])

Epoch 1/50
18/18 [==============================] - 2s 50ms/step - loss: 0.6918 - auc: 0.5010 - val_loss: 0.6406 - val_auc: 0.5074
Epoch 2/50
18/18 [==============================] - 1s 34ms/step - loss: 0.6111 - auc: 0.5310 - val_loss: 0.5592 - val_auc: 0.5603
Epoch 3/50
18/18 [==============================] - 1s 30ms/step - loss: 0.5373 - auc: 0.5746 - val_loss: 0.4736 - val_auc: 0.6553
Epoch 4/50
18/18 [==============================] - 1s 29ms/step - loss: 0.4689 - auc: 0.6368 - val_loss: 0.4055 - val_auc: 0.7288
Epoch 5/50
18/18 [==============================] - 1s 30ms/step - loss: 0.4193 - auc: 0.6867 - val_loss: 0.3662 - val_auc: 0.7631
Epoch 6/50
18/18 [==============================] - 1s 31ms/step - loss: 0.3952 - auc: 0.7112 - val_loss: 0.3477 - val_auc: 0.7794
Epoch 7/50
18/18 [==============================] - 1s 32ms/step - loss: 0.3790 - auc: 0.7333 - val_loss: 0.3389 - val_auc: 0.7890
Epoch 8/50
18/18 [==============================] - 1s 31ms/step - loss: 0.3719 - a

experimenting with weights initialization

In [ ]:
from keras.initializers import he_normal

In [ ]:
# Create a Sequential model
model = Sequential()

# Add layers to the model (example architecture with dropout)
model.add(Dense(20, input_dim=21, activation='relu', kernel_initializer=he_normal()))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.5
model.add(Dense(10, input_dim=21, activation='relu'))
model.add(Dropout(0.25))  # Example dropout layer with a dropout rate of 0.5
model.add(Dense(1, activation='sigmoid'))
# Create an SGD optimizer with momentum
sgd_optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC']) #change optimizer
model.fit(trainX, trainy, validation_data=(valX, valy), epochs=50, batch_size=2048, callbacks=[early_stopping])